In [ ]:
# | default_exp _cli

In [ ]:
# | export

import importlib
import sys
import asyncio
from pathlib import Path
from typing import *
import signal
from os import getpid
import time
import anyio
import threading
import copy
from contextlib import contextmanager
import multiprocessing
import subprocess
from http.server import HTTPServer, SimpleHTTPRequestHandler
import socketserver

import typer
from fastapi import FastAPI

from fastkafka.application import FastKafka
from fastkafka._components.logger import get_logger, supress_timestamps
from fastkafka._components.helpers import _import_from_string
from fastkafka.server import run_fastkafka_server, terminate_asyncio_process, run_in_process
from fastkafka.testing import change_dir

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory

import nbformat
from nbconvert import PythonExporter
from typer.testing import CliRunner

from fastkafka.testing import change_dir
from fastkafka.server import generate_app_in_tmp

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")
_run_app = typer.Typer(help="")
#_app.add_typer(_run_app, name="run")
_docs_app = typer.Typer(help="Collection of functions for managing fastkafka app documentation")
_app.add_typer(_docs_app, name="docs")

@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(multiprocessing.cpu_count(), help="Number of FastKafka instances to run, defaults to number of CPU cores."),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        asyncio.run(run_fastkafka_server(num_workers=num_workers, app=app))
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
! nbdev_export

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
                "fastkafka", "run", app, stdout=asyncio.subprocess.PIPE
            )
    time.sleep(5)
    proc.terminate()
    await proc.wait()
    outputs, errs = await proc.communicate()
    
    print(outputs.decode("utf-8"))
    assert proc.returncode == 0, proc.returncode

In [ ]:
# | export

@_docs_app.command(
    "generate",
    help="Creates documentation for a Fast Kafka API application ",
)
def generate_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be created"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.skip_docs = False
        application.create_docs()
    except Exception as e:

        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

@_docs_app.command(
    "serve",
    help="Creates documentation for a Fast Kafka API application ",
)
def serve_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be created"
    ),
    bind: str = typer.Option(
        "0.0.0.0", help="Some info"
    ),
    port: int = typer.Option(
        8000, help="Some info"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.skip_docs = False
        application.create_docs()
        with change_dir("asyncapi/docs/"):
            server_address = (bind,  port)
            handler = SimpleHTTPRequestHandler
            
            #httpd = HTTPServer(server_address, handler)
            
            d = {"should_stop": False}
            
            def sigint_handler(signal, frame, d:Dict[str, bool]=d):
                d["should_stop"] = True
            
            signal.signal(signal.SIGINT, sigint_handler)
            signal.signal(signal.SIGTERM, sigint_handler)
            
            with socketserver.TCPServer(server_address, handler) as httpd:
                httpd.timeout = 0.1
                typer.secho(f"Serving documentation on http://{server_address[0]}:{server_address[1]}")
                while not d["should_stop"]:
                    httpd.handle_request()
                typer.secho(f"Interupting serving of documentation and cleaning up...")
    except Exception as e:

        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
! nbdev_export

In [ ]:
result = runner.invoke(_app, ["docs", "generate", "--help"])

In [ ]:
with generate_app_in_tmp() as import_str:
        result = runner.invoke(_app, ["docs", "generate", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

        result = runner.invoke(_app, ["docs", "generate", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

In [ ]:
result = runner.invoke(_app, ["docs", "serve", "--help"])

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
                "fastkafka", "docs", "serve",  "--port=48000", app, stdout=asyncio.subprocess.PIPE
            )
    time.sleep(30)
    proc.terminate()
    await proc.wait()
    outputs, errs = await proc.communicate()
    assert proc.returncode == 0, proc.returncode
    print(outputs.decode("utf-8"))